In [4]:
import tensorflow as tf
import numpy as np
from datetime import datetime
import os
print(tf.__version__)

2.0.0


In [2]:
class MNISTLoader():
    def __init__(self):
        mnist = tf.keras.datasets.mnist
        (self.train_data, self.train_label), (self.test_data, self.test_label) = mnist.load_data()
        # MNIST中的图像默认为uint8（0-255的数字）。以下代码将其归一化到0-1之间的浮点数，并在最后增加一维作为颜色通道
        self.train_data = np.expand_dims(self.train_data.astype(np.float32) / 255.0, axis=-1)   #[60000, 28, 28, 1]
        self.test_data = np.expand_dims(self.test_data.astype(np.float32) / 255.0, axis=-1)   #[10000, 28, 28, 1]
        self.train_label = self.train_label.astype(np.int32)  #[60000]
        self.test_label = self.test_label.astype(np.int32)   #[10000]
        self.num_train_data, self.num_test_data = self.train_data.shape[0], self.test_data.shape[0]
        self.train_dataset = tf.data.Dataset.from_tensor_slices((self.train_data, self.train_label))
        self.train_dataset = self.train_dataset.shuffle(buffer_size=23000)
        self.train_dataset = self.train_dataset.batch(batch_size)
        self.train_dataset = self.train_dataset.prefetch(tf.data.experimental.AUTOTUNE)
        self.test_dataset = tf.data.Dataset.from_tensor_slices((self.test_data, self.test_label))
        self.test_dataset = self.test_dataset.shuffle(buffer_size=23000)
        self.test_dataset = self.test_dataset.batch(batch_size)
        self.test_dataset = self.test_dataset.prefetch(tf.data.experimental.AUTOTUNE)
        
class MLP(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.flatten = tf.keras.layers.Flatten()
        self.dense1 = tf.keras.layers.Dense(units=200, activation=tf.nn.relu, kernel_regularizer=tf.keras.regularizers.l2(l=0.01))
        self.bn1 = tf.keras.layers.BatchNormalization()
        self.dense2 = tf.keras.layers.Dense(units=200, activation=tf.nn.relu, kernel_regularizer=tf.keras.regularizers.l2(l=0.01))
        self.bn2 = tf.keras.layers.BatchNormalization()
        self.dense3 = tf.keras.layers.Dense(units=10, activation=tf.nn.softmax)
  
    def call(self, inputs, training=None):
        x = self.flatten(inputs)
        x = self.dense1(x)
        x = self.bn1(x, training=training)
        x = self.dense2(x)
        x = self.bn2(x, training=training)
        output = self.dense3(x)
        return output

class MLP_no_bn(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.flatten = tf.keras.layers.Flatten()
        self.dense1 = tf.keras.layers.Dense(units=200, activation=tf.nn.relu, kernel_regularizer=tf.keras.regularizers.l2(l=0.01))
        self.dense2 = tf.keras.layers.Dense(units=200, activation=tf.nn.relu, kernel_regularizer=tf.keras.regularizers.l2(l=0.01))
        self.dense3 = tf.keras.layers.Dense(units=10, activation=tf.nn.softmax)
  
    def call(self, inputs, training=None):
        x = self.flatten(inputs)
        x = self.dense1(x)
        x = self.dense2(x)
        output = self.dense3(x)
        return output

num_epochs = 50
batch_size = 128
learning_rate = 0.001

def train(train_dataset, test_dataset):
    history = model.fit(
         train_dataset,
         epochs=num_epochs,
         verbose=1,
         validation_data=test_dataset,
         callbacks=[cp_callback_mc]
    )
    return history
    
def train_generator(train_data, train_label, test_dataset):
    datagen = tf.keras.preprocessing.image.ImageDataGenerator()    
    datagen.fit(train_data)

    history = model.fit_generator(
         datagen.flow(train_data, train_label, batch_size=batch_size),
         steps_per_epoch=len(train_data) / batch_size,
         epochs=num_epochs,
         verbose=1,
         validation_data=test_dataset,
         callbacks=[cp_callback_mc]
         )
    return history

def test():
    loss_and_metrics = model.evaluate(test_dataset)
    print(loss_and_metrics)
    
if __name__ == '__main__':
    checkpoint_path = "training_bn_sample_1/cp-{epoch:04d}.ckpt"
    checkpoint_dir = os.path.dirname(checkpoint_path)

    #model = MLP()
    model = MLP_no_bn()
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                    loss=tf.keras.losses.sparse_categorical_crossentropy,
                    metrics=[tf.keras.metrics.sparse_categorical_accuracy])
    
    data_loader = MNISTLoader()
    
    cp_callback_mc = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                        save_weights_only=True,
                                                        verbose=0)
    #latest = tf.train.latest_checkpoint(checkpoint_dir)
    #if(latest != None):
        #model.load_weights(latest) 
    

In [ ]:
    start = datetime.now()
    history = train(data_loader.train_dataset, data_loader.test_dataset)
    end = datetime.now()
    print(end - start)

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline

train_loss=history.history['loss']
val_loss=history.history['val_loss']
train_acc=history.history['sparse_categorical_accuracy']
val_acc=history.history['val_sparse_categorical_accuracy']

epochs = range(len(train_acc))

plt.plot(epochs,train_loss,'r', label='train_loss')
plt.plot(epochs,val_loss,'b', label='val_loss')
plt.title('train_loss vs val_loss')
plt.xlabel('epoch')
plt.ylabel('loss')
plt.legend()
plt.figure()

plt.plot(epochs,train_acc,'r', label='train_acc')
plt.plot(epochs,val_acc,'b', label='val_acc')
plt.title('train_acc vs val_acc')
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.legend()
plt.figure()

In [ ]:
    train_generator(data_loader.train_data, data_loader.train_label, data_loader.test_dataset):